In [19]:
from margprob import *
from similarityfunctions import *
import itertools as it
import portion

# General

In [20]:
def CPT(graph, values):
    nodes = list(graph.nodes())
    CPT = list(it.product(values, repeat=len(nodes)))
    return CPT

def possible_outcomes(graph, values):
    nodes = list(graph.nodes())
    cpt = CPT(graph, values)
    list_outcomes = []
    for i in cpt:
        list_outcomes.append(dict(zip(nodes, i)))
    return list_outcomes

In [21]:
#find all observed (weighted) nodes in a graph
def getObservedNodes(dg):
    """accepts a graph, returns a list of the observed nodes in the graph"""
    observed_nodes = []
    dict_values = nx.get_node_attributes(dg, 'value')
    
    for v in dict_values:
        if (dg.nodes[v]['value'] == True):
            observed_nodes.append(v)
            
    return observed_nodes

# find all unobserved nodes
def getUnobservedNodes(dg):
    """accepts graph, returns list of False nodes"""
    unobserved_nodes = []
    dict_values = nx.get_node_attributes(dg, 'value')
    
    for v in dict_values:
        if (dg.nodes[v]['value'] == False): # and (dg.nodes[v]['type'] == bool)
            unobserved_nodes.append(v)

# Sum-Product Marginal Approximation

In [22]:
def approx_node_weight(parent, child, conditional_prob):
    '''Accepts a conditional probability value P(A|B), calculates possible probabilities of the adjacent node'''
    #instantiate root nodes
    if (dg.in_degree(parent) == 0): parent_prob = .9
    else: parent_prob = random.uniform(0,1)
    
    #instantiate terminal nodes
    if (dg.out_degree(child) == 0): child_prob = 1
    else: child_prob = random.uniform(0,1)
        
    likelihood = (conditional_prob*child_prob)/parent_prob
    known_conditional_prob = (parent_prob*likelihood)/child_prob
    
    if (known_conditional_prob != conditional_prob):
        return approx_node_weight(parent, child, conditional_prob)
    else: 
        dg.nodes[parent]['weight'] = parent_prob
        dg.nodes[child]['weight'] = child_prob
        return parent_prob, child_prob

In [23]:
# assign intrinsic probability to root and terminal nodes
def gen_node_weights(dg):
    '''Accepts a conditional probability value P(A|B), returns P(B|A) and a pair of values for P(A) and P(B) that
        would be consistent with the given conditional probability for each node in the graph'''
    nodes = list(dg.nodes())
    edges = list(dg.edges())
    roots = [root for root in nodes if (dg.in_degree(root) == 0)]
    terminal_nodes = [t for t in list(set(nodes) - set(roots)) if (dg.out_degree(t) == 0)] 

    unknown_nodes = list(set(roots) - set(terminal_nodes))
    for node in nodes:
        weight_approx = []
        for edge in edges:
            conditional_prob = dg[edge[0]][edge[1]]['weight']
            node_prob = approx_node_weight(edge[0], edge[1], conditional_prob)
            if (node == edge[0]): weight_approx.append(node_prob[0])
            else: weight_approx.append(node_prob[1])
        dg.nodes[node]['weight'] = mean(weight_approx)
        
    return dg.nodes()

In [24]:
# pick a node, calculate the marg prob
def margProb(dg, outcome, node):
    parents = list(dg.predecessors(node))
    #assign value to each node
    for n in list(dg.nodes()):
        dg.nodes[n]['value'] = outcome.get(n)
        if (dg.nodes[n]['value'] == False): dg.nodes[n]['weight'] = 1 - dg.nodes[n]['weight']

    node_value = dg.nodes[n]['value']
    node_prob = dg.nodes[n]['weight']

    #calculate prob of node given parents
    cp = []
    if not parents:
        margprob = node_prob
    else:
        for p in parents:
            edge_weight = dg[p][node]['weight']
            cp += [edge_weight]
        cp.append(node_prob)
        margprob = multiplyList(cp)
    
    return outcome, margprob
    

In [ ]:
def rootMargProb(dg, outcome, root_node):
    all_nodes = list(dg.nodes)
    all_ancestors = [node for node in all_nodes if (node in list(nx.ancestors(dg, root_node)))]
    
    dg.nodes[root_node]['value'] = outcome.get(root_node)
    if (dg.nodes[root_node]['value'] == False): dg.nodes[root_node]['weight'] = 1 - dg.nodes[root_node]['weight']
    root_prob = dg.nodes[root_node]['weight']
    
    prob_list = [root_prob]
    for a in all_ancestors:
        # for root's direct children
        if a in list(dg.successors(root_node)):
            cpd.append(dg[root_node][a]['weight'])
        else: 
            prob_list.append(margProb(dg, outcome, a))
        
    rootProb = multiplyList(prob_list)
    return outcome, rootProb
    

In [25]:
# pick a node, calculate the marg prob
def total_marginal_calculation(dg, node):
    worlds = possible_outcomes(dg, [True, False])
    marg_dist = []
    
    if node in [r for r in list(dg.nodes()) if (dg.in_degree(r) == 0) and dg.successors(r)]:
        for w in worlds:
            nodeMarg = rootMargProb(dg, w, node)
            marg_dist.append(nodeMarg)
        return marg_dist
    else:
        for w in worlds:
            nodeMarg = margProb(dg, w, node)
            marg_dist.append(nodeMarg)
        return marg_dist

# Tessam Interval-Valued Marginal Approximation

In [26]:
def getSubset(dg, observed_node):
    """accepts a graph and node name, returns a subgraph of that node's neighbors and non-d-separated nodes
        (includes the original node also)"""
    #get all neighbors
    children = list(dg.successors(observed_node))
    parents = list(dg.predecessors(observed_node))
    
    #get all non-d-separated nodes
    non_d_separated = []
    for n in list(dg.nodes()):
        n_children = list(dg.predecessors(n))
        commonDescendants = list(set(children) & set(n_children))
        if commonDescendants:
            non_d_separated += commonDescendants
            
    node_subset = [observed_node] + children + parents + commonDescendants

In [27]:
# pick a node, calculate the min marg prob
def margParents(dg, node):
    parents = list(dg.predecessors(node))
    
    observed_nodes = getObservedNodes(subdg)
    observed_evidence = list(set(parents) & set(observed_nodes))
    
    false_nodes = getUnobservedNodes(subdg)
    false_evidence = list(set(parents) & set(false_nodes))
    
    #calculate prob of node given parents
    if not parents:
        margprob = 0
    else:
        cp = []
        for p in parents:
            weight = dg[p][node]['weight']
            cp.append(weight)

        margprob = numpy.prod(cp)
    return margprob

In [28]:
def minMargInterval(dg, values, node):
    subdg = getSubset(dg, node)
    worlds = possible_outcomes(subdg, values)
    
    sub_nodes = list(subdg.nodes())
    parents = list(subdg.predecessors(node))
    ancestors = list(set(nx.ancestors(subdg, node)) - set(parents))
    
    margprobs = []
    minMarg = []
    for w in worlds:
        
        for node in sub_nodes:
            node_value = w.get(node)
            subdg.nodes[node]['value'] = node_value
        
        marg1 = margParents(subdg, node)
        margprobs.append(marg1)

        marg2 = [margParents(dg, p) for p in parents]
        marg3 = [margParents(dg, a) for a in ancestors]
        margprobs + marg2 + marg3
        
        node_marg = numpy.prod(margprobs)
        minMarg.append(node_marg)
        
    #Lessam_min = sum(minMarg)
    return numpy.sum(minMarg)
            

In [29]:
# pick a node, calculate the min marg prob
def margChildren(dg, node):
    children = list(dg.successors(node))
    
    #calculate prob of children given node
    if not children:
        margprob = 0
    else:
        cp = []
        for c in children:
            weight = dg[node][c]['weight']
            cp.append(weight)

        margprob = numpy.prod(cp)
    return margprob

In [30]:
def maxMargInterval(dg, values, node):
    subdg = getSubset(dg, node)
    worlds = possible_outcomes(subdg, values)
    
    sub_nodes = list(subdg.nodes())
    children = list(subdg.predecessors(node))
    descendants = list(set(nx.descendants(subdg, node)) - set(children))
    
    margprobs = []
    maxMarg = []
    for w in worlds:
        for node in sub_nodes:
            node_value = w.get(node)
            subdg.nodes[node]['value'] = node_value
            
        marg1 = margChildren(subdg, node)
        margprobs.append(marg1)

        marg2 = [margChildren(dg, c) for c in children]
        marg3 = [margChildren(dg, d) for d in descendants]
        margprobs + marg2 + marg3
        node_marg = numpy.prod(margprobs)
        
        maxMarg.append(node_marg)
        
    #Lessam_min = sum(minMarg)
    return numpy.sum(maxMarg)
            

In [31]:
# returns the interval-valued marginal probability of a hidden node 
def TessamInterval(dg, values, node):
    lower_bound = minMargInterval(dg, values, node)
    upper_bound = maxMargInterval(dg, values, node)
    marg_prob_interval = [lower_bound, upper_bound] #portion.closed(lower_bound, upper_bound)
    return marg_prob_interval
